# 朝倉書店「医学への統計学」7章 頻度に関する推測

## 7.2　母比率$p$に関する推測

In [4]:
using HypothesisTests
using Distributions

function binomial_test(r, n, p0, α)
    phat = r　/　n
    diff = abs(phat - p0) - 1/(2*n)
    σ = √((p0*(1 - p0))/n)
    z = diff / σ
    pval = cquantile(Normal(0, 1), α/2)
    lower = phat - pval * √((phat*(1 - phat))/n)
    upper = phat + pval * √((phat*(1 - phat))/n)
    @show z, pval
    @show lower, upper
end

r = 68; n = 635; p0 = 0.082; α = 0.05
binomial_test(r, n, p0, α)
;

(z, pval) = (2.2317769951683486, 1.9599639845400592)
(lower, upper) = (0.08303559475913208, 0.1311376335873246)


## 7.3　二つの母比率の差の検定と信頼区間

In [5]:
function diff_binomial_test(r1, r2, n1, n2, α)
    phat1 = r1 / n1
    phat2 = r2 / n2
    pbar = (r1 + r2) / (n1 + n2)
    qbar = 1 - pbar
    diff = abs(phat1 - phat2) - 0.5*(1/n1 + 1/n2)
    σ = √(pbar * qbar * (1/n1 + 1/n2))
    z = diff / σ
    pval = cquantile(Normal(0, 1), α/2)
    lower = phat1 - phat2 - pval * √((phat1*(1-phat1))/n1 +(phat2*(1-phat2))/n2)
    upper = phat1 - phat2 + pval * √((phat1*(1-phat1))/n1 +(phat2*(1-phat2))/n2)
    @show z, pval
    @show lower, upper
end

r1 = 29; r2 = 10; n1 = 109; n2 = 104; α = 0.05
diff_binomial_test(r1, r2, n1, n2, α)
;

(z, pval) = (3.027645444909852, 1.9599639845400592)
(lower, upper) = (0.06944234274764742, 0.27036005668777957)


## 7.4　2 x 2 分割表の$\chi^2$検定

In [6]:
function chi_squared_test(a, b, c, d, α)
    n1 = a + b
    n2 = c + d
    m1 = a + c
    m2 = b + d
    N = n1 + n2
    tmp = N * (abs(a*d - b*c) - 0.5*N)^2
    chi_squared = tmp / (n1*n2*m1*m2)
    pval = cquantile(Chisq(1), α)
    @show chi_squared, pval
end

a = 29; b = 109 - 29; c = 10; d = 104 - 10; α = 0.01
chi_squared_test(a, b, c, d, α)
;

(chi_squared, pval) = (9.166636940083375, 6.634896601021213)


## 7.5　Fisherの正確な検定

In [11]:
function fisher_exact_test(a, n1, n2, m1, m2)
    b = n1 - a
    c = m1 - a
    d = n2 - c
    N = n1 + n2
    prob = (binomial(m1,a) * binomial(m2,b)) / binomial(N,n1)
    @show a, prob
end

as = 0:5; n1 = 19; n2 = 9; m1 = 5; m2 = 23
for a in as
    fisher_exact_test(a, n1, n2, m1, m2)
end
;

(a, prob) = (0, 0.001282051282051282)
(a, prob) = (1, 0.02435897435897436)
(a, prob) = (2, 0.14615384615384616)
(a, prob) = (3, 0.35494505494505496)
(a, prob) = (4, 0.35494505494505496)
(a, prob) = (5, 0.11831501831501831)


In [19]:
n1 = 19; n2 = 9; m1 = 5; m2 = 23
as = 0:1
bs = n1 .- as
cs = m1 .- as
ds = n2 .- cs
N = n1 + n2
prob = ones(length(as))
for i in length(prob)
    prob[i] = (binomial(m1,as[i]) * binomial(m2,bs[i])) / binomial(N,n1)
end
prob

2-element Vector{Float64}:
 1.0
 0.02435897435897436

In [ ]:
function mid_pval_fisher_exact_test(as, n1, n2, m1, m2)
    bs = n1 - a
    c = m1 - a
    d = n2 - c
    N = n1 + n2
    prob = ones(length(a))
    for i in length(prob)
        for a in as
            prob[i] = (binomial(m1,a) * binomial(m2,b)) / binomial(N,n1)
        end
    end
    one_sided_mid_pval = prob[] + 0.5*prob[]
    two_sided_mid_pval = one_sided_mid_pval * 2
    @show one_sided_mid_pval
    @show two_sided_mid_pval 
end

## 7.6　対応のある場合の二つの母比率の差の検定と信頼区間　

In [ ]:
function pair_diff_binomial_test(a, b, c, d, α)
end



## 7.7　適合度の$\chi^2$検定

## 7.8　罹患率、死亡率に関する推測

## 7.9　罹患率、死亡率の差と比に関する推測

## 7.10　疫学研究と疾病発生リスクの比較指標

## 7.11　順序カテゴリー変数に基づく2群の差のWilcoxonの順位和検定